In [1]:
#!pip3 install -r ../requirements.txt

#### Challenge - David Felipe Martinez Castiblanco

## Generalidades

A continuación, presentaré una serie de generalidades que aplican para todas las preguntas:

##### Estilo de código
  - Usé pylint y pycodestyle para aumentar la calidad de mi código, sin embargo desactivé dos reglas:
     - R0801: Existe código duplicasdo entre archivos, la razón es que estoy usando estrategias análogas en los archivos, soy conciente que es una mala practica y el código deberia ser mucho más modular, sin embargo etendí que el ejercicio no lo permite.
     - W1514: Estoy abriendo archivos sin especificar su codificación, la razón de desactivarlo es que no conozco la codificación original del archivo, y no quiero depronto dañar mis calculos por una mala codificació, esta vez le dejo a python buscar la correcta.

##### Estructura del Dataset:
  - Es un dataset excesivamente grande en comparación a lo que se requiere de este. Solamente se usaron 3 atributos de todos los que tiene cada muestra.
  - Debido a que los atributos requeridos no tenian inconsistencias, no se hizo limpieza alguna del dataset, exceptuando la limpieza minima de cada ejercicio (pasar a mayusculas un usuario, extraer substrings de algún texto).
  - Vi que varios tweets tenian otros tweets anidados (quoted), sin embargo hice la suposición de que no se tendrán en cuenta para los conteos, es decir, no están incluidos en ninguna parte de mi ejercicio.

##### Lectura de los datos:

  - Estamos hablando de un archivo algo pesado. Si pensamos en cargarlo por completo en memoria, tendríamos que aprovecharlo muy bien y empezar a paralelizar tareas. Este sería el caso cuando queremos optimizar el tiempo de ejecución.
  - Si tenemos los datos divididos desde un principio, podríamos hacer que múltiples hilos abran cada partición y la procesen. En nuestro caso, tenemos un archivo completo, pero podemos cargarlo todo en memoria, particionarlo en chunks y hacer cálculos de manera paralela.
     - Entendí que el ejercicio no me permite usar más funciones, pero en una futura mejora, veo obligatorio particionar el archivo y limpiarlo para dejar solo los atributos que se necesitan. Esto facilita y optimiza el proceso de trasnformación y obtención de metricas
  - En los 3 ejercicios enfocados en tiempo de procesamiento, los llevé hacia el enfoque map-reduce, y por medio de multihilo procesé lo que se requería. Sé que no es tan óptimo cuando se presentan pocos datos o un solo archivo, pero cuando la cantidad de datos tiende crece de manera masiva(big data), esta solución superaría por mucho al enfoque que di optimizando memoria.
  - Por el contrario, si necesitamos optimizar el uso de memoria, podemos ir leyendo los datos línea a línea y calculando secuencialmente lo que se requiere. Esto se convierte en una tarea lineal, pero es una manera de cargar un archivo pesado optimizando el uso de memoria.
  - Está claro que al escalar el problema, es decir, si tenemos x archivos del mismo o mayor tamaño, esta solución se vuelve insostenible. Por lo que debe escalar hacia algo enfocado en arquitectura big data. Python, por ejemplo, nos ofrece librerías como MRJob para esto.
  - Siempre es útil liberar memoria dinamicamente, es decir, borrar aquellas variables pesadas que se usan una sola vez.
  - A gran escala, el cargue de los datos es tan importante como su procesamiento, y la nube también nos ofrece herramientas para lidiar con ello. Herramientas como AWS Glue para jobs basados en Spark, S3 para almacenamiento, Kinesis para streaming y analytics, etc.


##### Estructuras de datos, funciones integradas y librerias:

  - Python ofrece una amplia variedad de funciones integradas que están altamente optimizadas, por lo que es una buena práctica usarlas en cada escenario pertinente.
  - Para los problemas presentados en este challenge, me pareció muy pertinente usar funciones como map(), reduce(), defaultdict(), Counter(). Además, integrarlas con el uso de generadores, ya que me permite hacer un uso muy eficiente de memoria.
  - Existen librerias que facilitan muchisimo el trabajo y los calculos, sin embargo toca evaluar cuando es pertinente usar cada recurso. Como es el caso de pandas, que en la ultima sección hablaré de ello.

#### Q1

- Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.


   - Este problema tiene un grado de complejidad adicional, ya que se necesita un top_n anidado: primero uno para las fechas de los tweets y luego otro top_1 para el usuario de cada día.
   - Este es un problema bastante común y siempre estará ligado al problema de ordenamiento, por lo que la típica solución de recorrer los datos, ordenarlos y sacar los últimos elementos dependerá del algoritmo de ordenamiento usado.
   - En el enfoque de tiempo de ejecución, opté por implementar un mapper donde cada partición de datos termina en un top_10 local. Estos top_10 tienen anclados los usuarios que han interactuado cada día. Aquí debo decir que no es posible retornar un top_10 local y el top_1 calculado, ya que requiero la suma de todos los usuarios para poder hacer un buen reducer en el siguiente paso.
   - Como reducer, se reciben N top_10 locales y simplemente se juntan por la llave fecha, es decir, se reduce a un top_10.
   - Al final del algoritmo, cuando tengo solo 10 fechas, procedo a hacer el top_1 y retornar.
   - Note que el problema del ordenamiento sigue implícito, simplemente que al ordenar sobre arreglos más pequeños y de forma paralela, la complejidad alcanza a reducirse.
   - Como el ejercicio solo pide retornar la fecha junto con el usuario, me pareció que da un buen toque visual ordenar este top 10 por fecha.

In [2]:
# libraries to measure performance
from cProfile import Profile
from pstats import SortKey, Stats
from memory_profiler import memory_usage

# functions used on challenge
from q1_memory import q1_memory
from q1_time import q1_time
from q2_memory import q2_memory
from q2_time import q2_time
from q3_memory import q3_memory
from q3_time import q3_time

# general imports
from statistics import mean
import pprint
from datetime import datetime
import pandas as pd

# set file_path (same for all project)
file_path = "farmers-protest-tweets-2021-2-4.json"

In [3]:
with Profile() as profile:
    q1_memory_stat1 = memory_usage((lambda : pprint.pprint(q1_memory(file_path)),(), {}),interval=0.1)
    q1_memory_stat2 = Stats(profile)
    q1_memory_stat2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

print(f"\n\nMemory: {mean(q1_memory_stat1):.2f} MB")
print(f"Time: {sum(stat[2] for stat in q1_memory_stat2.stats.values()):.2f} Segs")

[(datetime.datetime(2021, 2, 12, 0, 0), 'RanbirS00614606'),
 (datetime.datetime(2021, 2, 13, 0, 0), 'MaanDee08215437'),
 (datetime.datetime(2021, 2, 14, 0, 0), 'rebelpacifist'),
 (datetime.datetime(2021, 2, 15, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 16, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 17, 0, 0), 'RaaJVinderkaur'),
 (datetime.datetime(2021, 2, 18, 0, 0), 'neetuanjle_nitu'),
 (datetime.datetime(2021, 2, 19, 0, 0), 'Preetm91'),
 (datetime.datetime(2021, 2, 20, 0, 0), 'MangalJ23056160'),
 (datetime.datetime(2021, 2, 23, 0, 0), 'Surrypuria')]


Memory: 104.28 MB
Time: 5.03 Segs


In [4]:
with Profile() as profile:
    q1_time_stat1 = memory_usage((lambda : pprint.pprint(q1_time(file_path)),(), {}),interval=0.1)
    q1_time_stat2 = Stats(profile)
    q1_time_stat2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

print(f"\n\nMemory: {mean(q1_time_stat1):.2f}MB")
print(f"Time: {sum(stat[2] for stat in q1_time_stat2.stats.values()):.2f} Segs")

[(datetime.datetime(2021, 2, 12, 0, 0), 'RanbirS00614606'),
 (datetime.datetime(2021, 2, 13, 0, 0), 'MaanDee08215437'),
 (datetime.datetime(2021, 2, 14, 0, 0), 'rebelpacifist'),
 (datetime.datetime(2021, 2, 15, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 16, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 17, 0, 0), 'RaaJVinderkaur'),
 (datetime.datetime(2021, 2, 18, 0, 0), 'neetuanjle_nitu'),
 (datetime.datetime(2021, 2, 19, 0, 0), 'Preetm91'),
 (datetime.datetime(2021, 2, 20, 0, 0), 'MangalJ23056160'),
 (datetime.datetime(2021, 2, 23, 0, 0), 'Surrypuria')]


Memory: 482.89MB
Time: 4.88 Segs


#### Q2

- Los top 10 emojis más usados con su respectivo conteo.


   - Este ejercicio también involucra obtener un top N, solo que este es más sencillo (no es anidado).
   - Como mencioné anteriormente, el problema de obtener un top N está muy relacionado con la complejidad en el ordenamiento.
   - La complejidad aquí radica en el hecho de reconocer emojis, ya que no es tarea fácil debido a la infinidad que existe. Sabemos que vienen codificados en Unicode y conocemos algunos, pero ¿cómo obtener absolutamente todos los emojis y no dejar ninguno por fuera?
     - Para esto me apoyé en la librería emoji, que es muy efectiva en el tratamiento de este tipo de caracteres. Aquí encontré que buscar emojis en un texto puede llegar a ser costoso, y que esta libreria lo hace mediante árboles.
     - Sin embargo, me pareció una buena oportunidad para probar el poder del paralelismo, ya que al ser una tarea más exhaustiva que los otros problemas, tener 2 o 3 particiones haciéndolo al mismo tiempo trae sus beneficios. Aquí vemos que el tiempo de ejecución es casi la mitad que el tiempo de ejecución lineal.
     - Es posible usar otra aproximación al problema y hacerlo aún más rápido, como utilizar diccionarios y expresiones regulares para reconocer los emojis. Sin embargo, podríamos estar perdiendo precisión en el conteo, ya que algunos emojis podrían no reconocerse.

In [5]:
file_path = "farmers-protest-tweets-2021-2-4.json"
with Profile() as profile:
    q2_memory_stat1 = memory_usage((lambda : pprint.pprint(q2_memory(file_path)),(), {}),interval=0.1)
    q2_memory_stat2 = Stats(profile)
    q2_memory_stat2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

print(f"\n\nMemory: {mean(q2_memory_stat1):.2f}MB")
print(f"Time: {sum(stat[2] for stat in q2_memory_stat2.stats.values()):.2f} Segs")

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]


Memory: 185.68MB
Time: 31.59 Segs


In [6]:
with Profile() as profile:
    q2_time_stat1 = memory_usage((lambda : pprint.pprint(q2_time(file_path)),(), {}),interval=0.1)
    q2_time_stat2 = Stats(profile)
    q2_time_stat2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)

print(f"\n\nMemory: {mean(q2_time_stat1):.2f}MB")
print(f"Time: {sum(stat[2] for stat in q2_time_stat2.stats.values()):.2f} Segs")

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]


Memory: 534.25MB
Time: 18.22 Segs


#### Q3
- El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

   - Este problema es muy análogo al anterior.
   - La complejidad aquí radica en reconocer los nombres de usuario para la plataforma de Twitter:
     - Entre 4 y 15 caracteres.
     - No reconoce entre mayúsculas y minúsculas.
     - Solo acepta caracteres alfanuméricos y guion bajo (en cualquier orden).
   - Me apoyé en el módulo integrado de Python "re", este módulo es increíblemente eficiente para trabajar con expresiones regulares.

In [7]:
file_path = "farmers-protest-tweets-2021-2-4.json"
with Profile() as profile:
    q3_memory_stat1 = memory_usage((lambda : pprint.pprint(q3_memory(file_path)),(), {}),interval=0.1)
    q3_memory_stat2 = Stats(profile)
    q3_memory_stat2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)
print(f"\n\nMemory: {mean(q3_memory_stat1):.2f}MB")
print(f"Time: {sum(stat[2] for stat in q3_memory_stat2.stats.values()):.2f} Segs")

[('NARENDRAMODI', 2266),
 ('KISANEKTAMORCHA', 1840),
 ('RAKESHTIKAITBKU', 1642),
 ('PMOINDIA', 1427),
 ('RAHULGANDHI', 1146),
 ('GRETATHUNBERG', 1048),
 ('RAVISINGHKA', 1019),
 ('RIHANNA', 986),
 ('UNHUMANRIGHTS', 962),
 ('MEENAHARRIS', 926)]


Memory: 194.82MB
Time: 6.10 Segs


In [8]:
with Profile() as profile:
    q3_time_stat1 = memory_usage((lambda : pprint.pprint(q3_time(file_path)),(), {}),interval=0.1)
    q3_time_stat2 = Stats(profile)
    q3_time_stat2.strip_dirs()
    #.sort_stats(SortKey.CUMULATIVE)
    #.print_stats(10)
print(f"\n\nMemory: {mean(q3_time_stat1):.2f}MB")
print(f"Time: {sum(stat[2] for stat in q3_time_stat2.stats.values()):.2f} Segs")

[('NARENDRAMODI', 2266),
 ('KISANEKTAMORCHA', 1840),
 ('RAKESHTIKAITBKU', 1642),
 ('PMOINDIA', 1427),
 ('RAHULGANDHI', 1146),
 ('GRETATHUNBERG', 1048),
 ('RAVISINGHKA', 1019),
 ('RIHANNA', 986),
 ('UNHUMANRIGHTS', 962),
 ('MEENAHARRIS', 926)]


Memory: 488.95MB
Time: 5.03 Segs


### Pandas

- Pandas es justamente una libreria totalmente optimizada para el manejo y transformación de datos, por esta razón quiero mostrar un pequeño ejemplo de como se solucionaria Q1 usando esta libreria.

- Adicional, quiero que veamos algunas consideraciones:
  - pandas es extremadamente habil haciendo transformación de datos gracias a su capacidad de tener todos los datos cargados en memoria sin importar si se está usando o no. Esto tiene vetajas como los 200ms obtenidos solamente al hacer la agrupación requerida en Q1.
  - El hecho de cargar todo en memoria lo hace muy optimo en comlejidad computacional, pero no lo hace nada optimo en uso de memoria, esto se refleja en el tiempo tan extenso que toma cargar todos los datos (400mb approx). Solamente cargando el dataset, demoramos 10 segundos, más que la solución completa presentada en la primera sección, y además, solo el dataset, si hacer transformaciones aún, consume approx 280Mb en memoria.
  - También es cierto que pandas nos deja hacer cargues a manera de batch, es decir, podemos partir este archivo en x dataframes durante el cargue, y esto optimiza mucho los tiempos, sin embargo no quita el hecho de que seguirá todo cargado en memoria.
  - Por otro lado, es nuestra tarea que apenas se carga el datframe, eliminar todo lo que no sea necesario, y dejar cargado lo mínimo posible.

In [9]:
%%time
# Leo el archivo, le indico que cada linea es un json, y que las fechas las deje en string
df = pd.read_json(file_path, lines=True, convert_dates=False)

# Selecciono solamente las dos columnas que necesito, lo demás se pueder ir para liberrar memoria
df_selected = df[['date', 'user']]

# Obtener información sobre el DataFrame
info = df.info(memory_usage='deep')

# Elimino el dataframe original, ya que es excesivamente grande
del df

# Aplico la transformación del usuario, obtengo el username y me deshago de todo lo dem
df_selected['user'] = df_selected['user'].apply(lambda x: x['username'])

# Aplico la transformación de la fecha, para quedarme solo con AAAA-MM-DD, esto facilita los agrupamientos
df_selected['date'] = df_selected['date'].apply(lambda x: x[:10])

# Imprimir la información
print(info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117407 entries, 0 to 117406
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   url              117407 non-null  object 
 1   date             117407 non-null  object 
 2   content          117407 non-null  object 
 3   renderedContent  117407 non-null  object 
 4   id               117407 non-null  int64  
 5   user             117407 non-null  object 
 6   outlinks         117407 non-null  object 
 7   tcooutlinks      117407 non-null  object 
 8   replyCount       117407 non-null  int64  
 9   retweetCount     117407 non-null  int64  
 10  likeCount        117407 non-null  int64  
 11  quoteCount       117407 non-null  int64  
 12  conversationId   117407 non-null  int64  
 13  lang             117407 non-null  object 
 14  source           117407 non-null  object 
 15  sourceUrl        116495 non-null  object 
 16  sourceLabel      116495 non-null  obje

In [10]:
%%time

# Primera Agrupación de conteo por fecha. Esto nos da el conteo de tweets cada día.
# Observe que se resetean los indices para que el agrupamiento de conteo se vuelva una columna llamada "count"
group = df_selected.groupby(["date"]).size().reset_index(name="count").nlargest(10, "count")

# Se filtra el df principal para deshacernos de las fechas que ya no están en el top
df_selected = df_selected[df_selected["date"].isin(group["date"])]
del group

# Segunda Agrupación de conteo por fecha y usuario al tiempo. Esto nos da el conteo de cada usuario en cada día.
# Observe que se resetean los idices para volver a un dataframe con solo dos indices, el agrupamiento de conteo se vuelve una columna llamada "count"
group2 = df_selected.groupby(["date","user"]).size().reset_index(name='count')

# Ya teniendo el conteo de cada usuario en cada día, volvemos a agrupar por fecha, esta vez quedadoos con aque usuario que tuvo más conteo, es decir, aquel que hizo más publicacioes cada día
group2 = group2.groupby("date").apply(lambda groupdate: groupdate.nlargest(1, 'count'))

# este agrupamiento nos da el usuario más activo cada dia, juto con el indice en el que se encontraba
group2 = group2.reset_index(drop=True)
group2
pprint.pprint([(datetime.fromisoformat(group2["date"][i]), group2["user"][i]) for i in range(len(group2))])

[(datetime.datetime(2021, 2, 12, 0, 0), 'RanbirS00614606'),
 (datetime.datetime(2021, 2, 13, 0, 0), 'MaanDee08215437'),
 (datetime.datetime(2021, 2, 14, 0, 0), 'rebelpacifist'),
 (datetime.datetime(2021, 2, 15, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 16, 0, 0), 'jot__b'),
 (datetime.datetime(2021, 2, 17, 0, 0), 'RaaJVinderkaur'),
 (datetime.datetime(2021, 2, 18, 0, 0), 'neetuanjle_nitu'),
 (datetime.datetime(2021, 2, 19, 0, 0), 'Preetm91'),
 (datetime.datetime(2021, 2, 20, 0, 0), 'MangalJ23056160'),
 (datetime.datetime(2021, 2, 23, 0, 0), 'Surrypuria')]
CPU times: user 155 ms, sys: 20.8 ms, total: 176 ms
Wall time: 175 ms


<timed exec>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
